In [1]:
import os
import torch

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
import cv2
from torchvision.models import swin_v2_b

In [2]:
md_df = pd.read_csv('data/spec_sample.csv')
md_df

FileNotFoundError: [Errno 2] No such file or directory: 'data/spec_sample.csv'

In [ ]:
class GalaxyZooDateset(torch.utils.data.Dataset):

    def __init__(self, csv_path='data/spec_sample.csv', img_path='data/3565489/images', include_images=False):

        self.csv_path = csv_path
        self.img_path = img_path
        self.include_images = include_images
        
        self.df = pd.read_csv(csv_path)
            
    def get_image_tensor(self, asset_id):       

        im = cv2.imread(f"{self.img_path}/{asset_id}.jpg")
        im = np.array(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), dtype=np.float32)
        im = torch.from_numpy(np.permute_dims(im, (2, 0, 1)))
        return im

    def __len__(self):

        return self.df.shape[0]

    def __getitem__(self, index):        
        
        sample = {
            'dr7objid': self.df['dr7objid'][index],
            'class': self.df['gz2class'][index],
            'specz': self.df['REDSHIFT'][index],
            #'swin_embeddings':, # TODO: Add swin embedding after running the model
        }

        if self.include_images == True:

            asset_id = self.df['asset_id'][index]
            sample['image'] = self.get_image_tensor(asset_id)

        return sample

In [ ]:
dataset = GalaxyZooDateset(include_images=True)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = input[0].detach()
    return hook


model = torch.hub.load("pytorch/vision", "swin_v2_t", weights="DEFAULT", progress=False)
model.head.register_forward_hook(get_activation('head'))

In [ ]:
for batch in tqdm(dataloader):
    # Training loop here
    pass